In [7]:
import pandas as pd
import numpy as np
import sys
import os
import random, string
import json
from pprint import pprint

In [2]:
sys.path.append('../../../../')

In [3]:
def iob_iobes(tags):
    """
    IOB -> IOBES
    """
    new_tags = []
    for i, tag in enumerate(tags):
        if tag == 'O':
            new_tags.append(tag)
        elif tag.split('-')[0] == 'B':
            if i + 1 != len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('B-', 'S-'))
        elif tag.split('-')[0] == 'I':
            if i + 1 < len(tags) and \
                    tags[i + 1].split('-')[0] == 'I':
                new_tags.append(tag)
            else:
                new_tags.append(tag.replace('I-', 'E-'))
        else:
            print(tag)
            raise Exception('Invalid IOB format!')
    return new_tags

In [4]:
def SL(in_file, out_folder):
    # get setence as tokenized words
    sentences = []
    labels = []
    with open(in_file, 'r') as f:
        lines = f.readlines()
        sentence = []
        label = []
        for line in lines:
            line = line.rstrip()
            # print(line)
            if line.split('\t')[0] == '-DOCSTART-':
                continue
            elif not line:
                # print('space: ', line)
                if sentence:
                    sentences.append(sentence)
                if label:
                    labels.append(label)
                sentence = []
                label = []
            else:
                sentence.append(line.split('\t')[0])
                label.append(line.split('\t')[-1])
    """for idx, label in enumerate(labels):
        labels[idx] = iob_iobes(label)"""
    sentences_file = os.path.join(out_folder, 'sentences.txt')
    labels_file = os.path.join(out_folder, 'labels.txt')
    id_file = os.path.join(out_folder, 'ID.txt')
    with open(sentences_file, 'a') as s_f, open(labels_file, 'a') as l_f, open(id_file, 'a') as i_f:
            for sentence, label in zip(sentences, labels):
                sentence_str = " ".join(w for w in sentence)
                label_str = " ".join(l for l in label)
                # generate a unique ID for each sentence
                id_str = ''.join(random.choices(string.ascii_letters + string.digits, k=16))

                i_f.write(id_str+'\n')
                s_f.write(sentence_str+'\n')
                l_f.write(label_str+'\n')

In [8]:
def read_json(json_file):
    with open(json_file) as f:
        data = json.load(f)
        return data

In [5]:
op_data = '../../../../src/ner/data/socialrom_iobes_id'
if not os.path.exists(op_data):
    os.makedirs(op_data)

In [6]:
json_file = '../../../../resources/data/NER/socialrom/en_pos_nerc2_pred_anon_beautified.json'

In [9]:
json_read = read_json(json_file)

In [13]:
len(json_read)

5887

In [16]:
json_read[2]

{'article_author': [{'article_author_id': '4l1M7Z2gqG',
   'article_author_name': 'WzeYW9DAlp'}],
 'article_source': 'cafemom',
 'article_text': "Where do you shop and what do you think is the cheapest place to go grocery shopping? I really am curious to know because I want to try some other stores. Super 1 is way too expensive so I always go to smith's grocery store. We don't have a super walmart yet or else I would happily go there. I just am tired of the store i go to and it doesnt have much of a selection. I rarely go to super 1 unless i see a really good sale. It seems like everything is alot more expensive there then at the store i go to. We have 2 grocery stores in our town and thats it. Super 1 and smiths. Smiths being the cheaper of the two. But there's a town about 15 miles away that has many many grocery stores. So what store do you think is cheapest for food type of groceries?? I wanna go grocery shopping but I want to go to a different store with more of a selection then w

In [6]:
in_file = '../../../../resources/data/NER/BC5CDR-IOBES/train.tsv'
out_folder = os.path.join(op_data, 'train')
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
SL(in_file, out_folder)

In [7]:
in_file = '../../../../resources/data/NER/BC5CDR-IOBES/devel.tsv'
out_folder = os.path.join(op_data, 'val')
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
SL(in_file, out_folder)

In [8]:
in_file = '../../../../resources/data/NER/BC5CDR-IOBES/test.tsv'
out_folder = os.path.join(op_data, 'test')
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
SL(in_file, out_folder)